<a href="https://colab.research.google.com/github/biplobsd/predict-future-sales-/blob/main/Predict_Future_Sales_(Alpha_test).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#@title Predict Future Sales
input_date = "2019-07-14" #@param {type:"date"}
import numpy as np
import tensorflow as tf
import pandas as pd
import joblib
import os
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler

MODEL = 'data/model.tflite'
SCALER = 'data/scaler.gz'
year, month, day = input_date.split('-')
INPUT = int(month+day)
os.makedirs('data', exist_ok=True)

if not os.path.exists(MODEL):
  !wget https://github.com/biplobsd/predict-future-sales-/releases/download/v0.2/model.tflite -O $MODEL
  clear_output()

if not os.path.exists(SCALER):
  !wget https://github.com/biplobsd/predict-future-sales-/releases/download/v0.2/scaler.gz -O $SCALER
  clear_output()


def lite_model(interpreter, inputs):
  interpreter.allocate_tensors()
  interpreter.set_tensor(interpreter.get_input_details()[0]['index'], inputs)
  interpreter.invoke()
  output = {}
  labels = interpreter.get_signature_list()['serving_default']['outputs']
  for i,v in enumerate(interpreter.get_output_details()):
    output[labels[i]] = interpreter.get_tensor(v['index'])[0][0]
  return output

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

scaler = joblib.load(SCALER)
interpreter = tf.lite.Interpreter(model_path=MODEL)
print(INPUT)
prediction = lite_model(interpreter, np.float32([[INPUT]]))
pr = pd.DataFrame([prediction])
pd.DataFrame(scaler.inverse_transform(pr.values), columns=pr.columns, index=pr.index)

714


,almond_croissant,americano,angbutter,berry_ade,cacao_deep,caffe_latte,cheese_cake,croissant,croque_monsieur,gateau_chocolat,jam,lemon_ade,mad_garlic,merinque_cookies,milk_tea,orange_pound,pain_au_chocolat,pandoro,plain_bread,tiramisu,tiramisu_croissant,vanila_latte,wiener
0,0.340239,0.154654,0.154654,0.154654,0.494893,0.092793,0.433032,0.123723,0.185585,0.154654,0.030931,0.030931,0.185585,0.061862,0.154654,0.061862,0.092793,0.123723,0.185585,0.123723,0.061862,0.030931,0.061862
